In [1]:
import os
import numpy as np

import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

from DeepAE import DeepAutoencoder
from lightning_helper import *

In [2]:
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
X = dataset.data.numpy()
dataset = torch.utils.data.TensorDataset(torch.vstack([ToTensor()(X[i]) for i in range(X.shape[0])]), 
                                         dataset.targets)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw/train-images-idx3-ubyte.gz to /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/pongpisit/Desktop/research/PDE-FIND/NN/MNIST/raw



In [3]:
# setup data
train_loader = utils.data.DataLoader(dataset, 
                                     num_workers=4)

In [4]:
autoencoder = DeepAutoencoder(dimensions=[28*28, 64, 3], activation_function=nn.ReLU())
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)

class LitAutoEncoder(LightningNetwork):
    def __init__(self, autoencoder, optimizer):
        super(LitAutoEncoder, self).__init__(network=autoencoder, 
                                             optimizer=optimizer)
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        x_hat = self.network(x)
        loss = nn.functional.mse_loss(x_hat, x)
        return loss

# init the autoencoder
autoencoder = LitAutoEncoder(autoencoder=autoencoder, optimizer=optimizer)

In [5]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=5)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/pongpisit/Desktop/research/PDE-FIND/NN/lightning_logs

  | Name    | Type            | Params
--------------------------------------------
0 | network | DeepAutoencoder | 101 K 
--------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
2023-03-26 21:02:47.528179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [6]:
# choose your trained nn.Module
encoder = autoencoder.network.encoder
fake_image_batch = torch.rand(4, 28*28).float()

encoder.eval()
with torch.no_grad():
    embeddings = encoder(fake_image_batch)
    print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-5.5383, -0.4144, -0.4049],
        [-5.7762, -0.1296, -0.0343],
        [-4.9970, -0.1716, -0.4624],
        [-4.3180, -0.4418, -1.1856]]) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
